In [73]:
# import tensorflow and numpy
import tensorflow as tf
import numpy as np

In [74]:
# setup the parameters
# number of different M's
gradients = 8
# number of input values
vals = 3
iterations = 5000
learning_rate = 0.1

# maximum answer to be learnt
max = gradients**2 + vals

# defining function to make training data
def training_data():
    
    n = 0
    rows = 1
    # array which we are using as our x values 
    # in equation of linear line, y = Mx + b
    # it includes 1 extra value as this will be used as our labal
    x = np.arange(vals+1).astype(np.int32)
    # empty array to write our training data to
    y = np.array([])
    
    # loop so it cycles through every gradient and adds on different b's
    for i in range(gradients):
        # we loop it so it also creates lines with b's
        # the maximum values will always be gradients^2
        for e in range(max-vals*i):
            
            # append to y, x*n + e, where n = M and e = b
            # reshape so that each time it appends it creates a new line
            y = np.append(y, x*n+e).reshape(rows,vals+1)
            
            # increase number of rows to reshape it
            rows += 1
        
        # increase gradient by 1
        n+=1
        
    # return the training data
    # and number of lines to learn
    return(y,np.size(y,0))

# print the training data
print(training_data())

(array([[  0.,   0.,   0.,   0.],
       [  1.,   1.,   1.,   1.],
       [  2.,   2.,   2.,   2.],
       ..., 
       [ 43.,  50.,  57.,  64.],
       [ 44.,  51.,  58.,  65.],
       [ 45.,  52.,  59.,  66.]]), 452)


In [75]:
# data = training data
# training lines = number of different lines
data,training_lines = training_data()

# the length is for when we convert the numbers into a binary array
# the array will be all zeros except one, which will be 1
# this will be the particle in this point in time
# each one is like a frame in a video
# this value is the size of the maximum value + 1
length = max+1

# the full length is the length of all the input frames stacked into one, 1d array
full_length = length*vals

# this function turns the data into the arrays explained above
def set_data():
    
    # there are two arrays, one for training data nad one for labels
    in_data = np.zeros([training_lines,vals,length])
    # the labels will be one-hot arrays
    lab = np.zeros([training_lines,1,length])
    
    # this sets the values specified in the training data to one
    for i in range(training_lines):
        # we need to set each individual input value
        for a in range(vals):
            # set the value to a 1
            in_data[i][a][data[i][a]] = 1
            
        # set the label value to a 1
        lab[i][0][data[i][vals]] = 1
        
    # here, we reshape it tto the full length 1d array
    in_data = in_data.reshape(training_lines,1,full_length)
    
    # return the data and labels
    return(in_data,lab)    

# print converted data
#print(set_data())

In [76]:
# we define the weights, biases and inputs
# this will be input training data
x = tf.placeholder(tf.float32, [None, full_length])
# weights and biases
W = tf.Variable(tf.zeros([full_length, length]))
b = tf.Variable(tf.zeros([length]))
# function which gives the out put of training data
y = tf.matmul(x, W) + b

# we will feed the labels in here
y_ = tf.placeholder(tf.float32, [None, length])

# configure the loss function, using cross entropy
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))
# define the optimizer
#train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)
train_step = tf.train.AdagradOptimizer(learning_rate).minimize(cross_entropy)


In [77]:
# I am using a GPU
# this line limits memory usage of the GPU to 0.4 when session is created
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.4)

# create interactive session using the GPU line for above
sess = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options)) 

# initialize variables
init = tf.initialize_all_variables()
sess.run(init)

# now we run the learning loop
for _ in range(iterations):

    # set training data and labels
    x_data, y_data = set_data()
    # only use one lines data
    # chooses line data by evenly spreading data over iterations
    x_data = x_data[_//(iterations/(training_lines-1.))]
    y_data = y_data[_//(iterations/(training_lines-1.))]
    
    # run the training optimizer
    sess.run(train_step, feed_dict={x: x_data, y_: y_data})

    # print steps and error 20 times in total
    if _ % (iterations/20) == 0:
        print 'step', _, 'out of', iterations
        print 'error =', sess.run(cross_entropy, feed_dict={x: x_data, y_: y_data})

Exception AssertionError: AssertionError("Nesting violated for default stack of <type 'weakref'> objects",) in <bound method InteractiveSession.__del__ of <tensorflow.python.client.session.InteractiveSession object at 0x7fad6d93d650>> ignored
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:28: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:31: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:19: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:20: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


step 0 out of 5000
error = 3.82726
step 250 out of 5000
error = 2.94674
step 500 out of 5000
error = 4.30975
step 750 out of 5000
error = 2.25513
step 1000 out of 5000
error = 3.50557
step 1250 out of 5000
error = 2.93316
step 1500 out of 5000
error = 3.13457
step 1750 out of 5000
error = 2.60563
step 2000 out of 5000
error = 3.47502
step 2250 out of 5000
error = 2.45028
step 2500 out of 5000
error = 3.20691
step 2750 out of 5000
error = 4.24775
step 3000 out of 5000
error = 3.03115
step 3250 out of 5000
error = 3.93519
step 3500 out of 5000
error = 2.88351
step 3750 out of 5000
error = 3.68361
step 4000 out of 5000
error = 2.83238
step 4250 out of 5000
error = 3.57551
step 4500 out of 5000
error = 2.82545
step 4750 out of 5000
error = 3.43611


In [89]:
# initialize array to record test values
test_line = np.array([])

print 'input values, the max answer is', max-1, 'and max gradient is', gradients-1
# loop to ask user for input values
for i in range(vals):
    print '\nvalue',i+1,'?'
    val = int(input())
    test_line = np.append(test_line, val)
    
# define function to convert test data into shape and form
# of training data
def test_model():
    
    # this is the array we are writing to
    input_array = np.array([])
    
    # for each input value, write to the input array
    # with a 1 in the position
    for a in range(vals):
        test_array = np.zeros([1,length])
        test_array[0][test_line[a]] = 1
        # we write to array and shape so new line for each test value
        input_array = np.append(input_array, test_array).reshape([1,(a+1)*length])
        
    # return array
    return(input_array)   

# print array
#test_model()

input values, the max answer is 66 and max gradient is 7

value 1 ?
4

value 2 ?
7

value 3 ?
10


In [90]:
# run the function to get output
# using new weights nad biases
probs = sess.run(y, feed_dict={x:test_model()})

# "true" answer, should work with all linear lines
answer = (test_line[1]-test_line[0])+test_line[vals-1]

if answer == np.argmax(probs):
    print 'Correct'
    
    # print learnt answer and it probability
    print '\nLearnt answer =', np.argmax(probs)
    print 'Probaility of', np.argmax(probs), '=', probs[0][np.argmax(probs)]
else:
    print 'Wrong'
    
    # print learnt answer and it probability
    print '\nLearnt answer =', np.argmax(probs)
    print 'Probaility of', np.argmax(probs), '=', probs[0][np.argmax(probs)]
    
    # print true answer and its probabilities
    print '\nNumerical answer =', answer
    print 'Probability of', answer, '=', probs[0][answer]
    

# print the probabilities
print '\n\n', probs

Correct

Learnt answer = 13
Probaility of 13 = 0.227527


[[-2.43150401 -2.43089914 -2.45777893 -2.28208876 -1.47944534 -2.3238368
  -2.29417443 -0.61541188 -2.38377047 -1.45296788 -0.61892581 -0.64483106
  -1.48791277  0.22752672 -1.56141245 -0.72349226 -1.24955654 -1.29813552
  -1.95056093 -0.46187204 -1.82984757 -1.15099359 -0.9884187  -1.67166162
  -1.62128758 -1.02713811 -1.59207892 -1.59119034 -1.59703135 -1.60325575
  -1.60747218 -1.59138656 -1.61348999 -1.6103518  -1.61771011 -1.62432182
  -1.621665   -1.63110733 -1.63466907 -1.63310277 -1.63866162 -1.64463305
  -1.62989402 -1.65096354 -1.65437484 -1.64905941 -1.65644336 -1.6631465
  -1.65816164 -1.66686332 -1.67091131 -1.66394722 -1.67231011 -1.67609763
  -1.65633488 -1.67305255 -1.66873622 -1.67283106 -1.67604983 -1.67173648
  -1.66732168 -1.65827727 -1.65025318 -1.64235377 -1.63479066 -1.61435723
  -1.74271464 -4.11043406]]


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:22: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
